<a href="https://colab.research.google.com/github/Vakhranev/Pushkina/blob/main/%D0%9C%D1%91%D1%80%D0%B4%D0%B6%20%D1%81%20%D0%B3%D0%BB%D0%BE%D1%81%D1%81%D0%B0%D1%80%D0%B8%D1%8F%D0%BC%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

# Загружаем файлы
merged_terms = pd.read_excel("merged_terms.xlsx")
glossary_terms = pd.read_excel("glossary_terms_cleaned.xlsx")

# Создаём список для результатов
final_rows = []

# Преобразуем список терминов глоссария в множество для быстрого поиска
glossary_set = set(glossary_terms["Термины глоссариев"].dropna().str.strip().str.lower())

# Проверяем каждый термин из merged_terms
for _, row in merged_terms.iterrows():
    term_found = None

    # Проверяем "Начальные формы"
    if isinstance(row["Начальные формы"], str) and row["Начальные формы"].strip().lower() in glossary_set:
        term_found = row["Начальные формы"].strip()

    # Проверяем "termin", если не нашли в "Начальные формы"
    elif isinstance(row["termin"], str) and row["termin"].strip().lower() in glossary_set:
        term_found = row["termin"].strip()

    # Добавляем строку в итоговый список
    final_rows.append({
        **row.to_dict(),
        "Термин глоссариев (совпадение)": term_found,
        "Классы глоссариев": row.get("Классы глоссариев", None)  # Сохраняем классы глоссариев
    })

# Добавляем термины из глоссариев, которые не были найдены в merged_terms
for _, row in glossary_terms.iterrows():
    term = row["Термины глоссариев"]
    if isinstance(term, str) and term.strip().lower() not in glossary_set:
        final_rows.append({
            "Начальные формы": None,
            "termin": None,
            "Термин глоссариев (совпадение)": term,
            "Классы глоссариев": row.get("Классы глоссариев", None)
        })

# Создаём итоговый DataFrame
final_df = pd.DataFrame(final_rows)

# Сохраняем результат в Excel
final_df.to_excel("final_merged_with_comparisons.xlsx", index=False)

print("Файл final_merged_with_comparisons.xlsx сохранён.")

Файл final_merged_with_comparisons.xlsx сохранён.


In [2]:
import pandas as pd

# Загружаем файл
glossary_terms = pd.read_excel("glossary_terms.xlsx")

# Приводим столбец "Классы глоссариев" к числовому типу
glossary_terms["Классы глоссариев"] = pd.to_numeric(glossary_terms["Классы глоссариев"], errors="coerce")

# Удаляем дубликаты, оставляя запись с минимальным классом
glossary_terms = glossary_terms.sort_values("Классы глоссариев").drop_duplicates(subset=["Термины глоссариев"], keep="first")

# Сохраняем очищенные данные
glossary_terms.to_excel("glossary_terms_cleaned.xlsx", index=False)

print("Файл glossary_terms_cleaned.xlsx сохранён.")

Файл glossary_terms_cleaned.xlsx сохранён.


In [7]:
import pandas as pd

# Загружаем файлы
merged_terms = pd.read_excel("merged_terms.xlsx")
glossary_terms = pd.read_excel("glossary_terms_cleaned.xlsx")

# Создаём пустой список для результатов
final_rows = []

# Обрабатываем каждый термин из столбца "Термины глоссариев"
for term in glossary_terms["Термины глоссариев"]:
    # Ищем совпадения с "Начальные формы"
    match_found = False
    for _, row in merged_terms.iterrows():
        # Проверяем, что в "Начальных формах" строка, перед тем как вызвать strip()
        if isinstance(row["Начальные формы"], str) and row["Начальные формы"].strip().lower() == term.strip().lower():
            final_rows.append({**row.to_dict(), "Термин глоссариев (совпало с Начальной формой)": term})
            match_found = True
            break

    # Если совпадений с "Начальные формы" не найдено, ищем в "termin"
    if not match_found:
        for _, row in merged_terms.iterrows():
            # Проверяем, что в "termin" строка, перед тем как вызвать strip()
            if isinstance(row["termin"], str) and row["termin"].strip().lower() == term.strip().lower():
                final_rows.append({**row.to_dict(), "Термин глоссариев (совпало с termin)": term})
                match_found = True
                break

    # Если совпадений нет, записываем термин на отдельной строке
    if not match_found:
        final_rows.append({"Термин глоссариев (нет совпадений)": term})

# Создаём итоговый DataFrame
final_df = pd.DataFrame(final_rows)

# Сохраняем результат в Excel
final_df.to_excel("final_merged_with_comparisons.xlsx", index=False)

print("Файл final_merged_with_comparisons.xlsx сохранён.")

Файл final_merged_with_comparisons.xlsx сохранён.


In [4]:
import pandas as pd

# Загружаем файлы
final_merged = pd.read_excel("final_merged_with_comparisons.xlsx")
glossary_terms = pd.read_excel("glossary_terms_cleaned.xlsx")

# Создаём словарь для быстрого поиска классов глоссариев по терминам
glossary_dict = glossary_terms.set_index("Термины глоссариев")["Классы глоссариев"].to_dict()

# Проходим по строкам final_merged
for idx, row in final_merged.iterrows():
    term_in_final = row["Термин глоссариев (совпадение)"]

    # Если термин найден и есть в glossary_terms
    if isinstance(term_in_final, str) and term_in_final.strip().lower() in glossary_dict:
        # Получаем соответствующий класс глоссария
        class_value = glossary_dict[term_in_final.strip().lower()]

        # Обновляем столбец "Классы глоссариев" в final_merged
        final_merged.at[idx, "Классы глоссариев"] = class_value

# Сохраняем обновлённый файл
final_merged.to_excel("final_merged_with_updated_classes.xlsx", index=False)

print("Файл final_merged_with_updated_classes.xlsx сохранён.")

Файл final_merged_with_updated_classes.xlsx сохранён.
